<a href="https://colab.research.google.com/github/lukigu/real-estate-prediction/blob/main/price_prediction_streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 20.9 MB/s eta 0:00:00


In [2]:
%%writefile app.py

import streamlit as st
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
import numpy as np

st.title("Predykacja cen nieruchomości")

# File uploader
uploaded_file = st.file_uploader("Prześlij plik csv z danymi:", type=["csv"])

if uploaded_file is not None:
    # Read CSV file
    price_data = pd.read_csv(uploaded_file, parse_dates=True, index_col=0)

    # Display the data
    st.write(price_data)



    #price_data = df[0]
    price_data["adj_cena"] = price_data["srednia cena"] / price_data["inflacja"] * 100
    price_data["nastepny_kwartal"] = price_data["adj_cena"].shift(-3)
    price_data.dropna(inplace=True)
    price_data["change"] = (price_data["nastepny_kwartal"] > price_data["adj_cena"]).astype(int)
    predictors = [ 'pkb', 'liczba studentow na 10tys','liczba sprzedanych lokali',
       'srednie zarobki w wojewodztwie', 'adj_cena']
    target = "change"

    st.dataframe(price_data)


    default_value = 20
    default_value1 = 4
    START = st.text_input("Ile kwartałów w przeszłość chcesz wykorzystać do predykcji", default_value)
    STEP = st.text_input("Ile kwaratłów w przyszłość chcesz przewidzieć", default_value1)
    START = int(START)
    STEP = int(STEP)
    STEP1 = STEP + 2
    params = {
        'objective': 'binary:logistic',  # Specify the objective as binary classification
        'n_estimators': 100,  # Number of trees in the ensemble
        'learning_rate': 0.1,  # Learning rate for boosting
        'max_depth': 3,  # Maximum tree depth
        'subsample': 0.8,  # Subsample ratio of the training instances
        'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
        'random_state': 1  # Random state for reproducibility
    }


    def predict(train, test, predictors, target):
        model = xgb.XGBClassifier(**params)
        model.fit(train[predictors], train[target])
        preds = model.predict(test[predictors])
        return preds

    def backtest(data, predictors, target):
        all_preds = []
        for i in range(START, data.shape[0], STEP):
            train = price_data.iloc[:i]
            test = price_data.iloc[i:(i+STEP)]
            all_preds.append(predict(train, test, predictors, target))

        preds = np.concatenate(all_preds)
        return preds, accuracy_score(data.iloc[START:][target], preds)

    def predict1(train, test, predictors, target):
        model = xgb.XGBClassifier(**params)
        model.fit(train[predictors], train[target])
        preds = model.predict(test[predictors])
        return preds

    def backtest1(data, predictors, target):
        all_preds = []
        for i in range(START, data.shape[0], STEP):
            train = price_data.iloc[:i]
            test = price_data.iloc[i:(i+STEP)]
            all_preds.append(predict1(train, test, predictors, target))
        preds = np.concatenate(all_preds)
        # Make prediction for one year
        one_year_data = data.iloc[-STEP1:]
        one_year_preds = predict1(data, one_year_data, predictors, target)
        return preds, accuracy_score(data.iloc[START:][target], preds), one_year_preds

    # Button
    button_clicked = st.button("Potwierdź")

    # Check if the button is clicked
    if button_clicked:

        preds, accuracy = backtest(price_data, predictors, target)

        pred_match = (preds == price_data[target].iloc[START:])
        pred_match[pred_match == True] = "green"
        pred_match[pred_match == False] = "red"

        # Generate scatter plot
        plot_data = price_data.iloc[START:].copy()
        fig, ax = plt.subplots()
        plot_data.reset_index().plot.scatter(x="rok", y="adj_cena", color=pred_match, ax=ax)
        plt.xticks(rotation=45, ha='right')

        # Display the plot in Streamlit
        st.pyplot(fig)

        st.write("Skuteczność predykcji:", accuracy)

        # Button
        #button_clicked1 = st.button("Sprawdź predykcje na przyszły rok")

        # Check if the button is clicked
        #if button_clicked1:

        preds1, accuracy1, one_year_preds = backtest1(price_data, predictors, target)

        QQQ = {'Q' : ['Q2', 'Q3', 'Q4', 'Q1','Q2', 'Q3']}

        col1, col2 = st.columns(2)
        with col1:
            st.write("Q")
            for q in QQQ['Q']:
                st.write(q)
        with col2:
            st.write("Cena w górę, czy w dół")
            for year in one_year_preds:
                st.write(year)


Writing app.py


In [3]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.161s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [4]:
!streamlit run app.py &>/content/logs.txt &

In [5]:
import socket
import urllib.parse
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.125.228.19
npx: installed 22 in 4.113s
your url is: https://little-dancers-rhyme.loca.lt
